# Double-DQN, Priority Experience Replay (PER) and Dueling-DQN 

Reference:
+ Dual DQN paper: https://arxiv.org/abs/1509.06461
+ Julien Blog: https://julien-vitay.net/deeprl/Valuebased.html#sec:double-dqn 
+ Efficient implement of Priority Experience Replay: https://jaromiru.com/2016/11/07/lets-make-a-dqn-double-learning-and-prioritized-experience-replay/
  

## I. Fixing problems of DQN:
### 1. Over-Estimate and Dual-DQN:
Recall the steps to update the target network in the original [DQN](Q-Learning.ipynb), 
+ S1: Randomly sample a batch of data from the buffer: $\{(s_i,a_i,r_i,s_{i+1})\}$.
+ S2: Compute the Target value $Q_{target}= r_i + \gamma Q_{\phi'}(s_{i+1},a')$, where $a'=\argmax Q_{\phi'}(s_{i+1},a')$.
+ S3: Compute the loss: $L=Mean(Q_{target} - Q_\phi(s_i,a_i))^2$, and perform SGD to update $\phi.

**Problem:** Note that in step S2, we use the same target network $Q_{\phi'}$ to (i) select the action $a'=\argmax Q_{\phi'}(s_{i+1},a')$ and (ii) update the target: $Q_{target}= r_i + \gamma Q_{\phi'}(s_{i+1},a')$. What 's the problem with it?
+ Chicken-Egg problem of Boostraping: We use an **estimated function $Q_{\phi'}$**  to **estimate the target** , and use this target to **update the estimation $Q_{\phi}$ again**. 
+ Especially at the beginning of learning when Q-values are far from being correct, because we select an action $a'$ that maximizes the estimated $Q_{\phi'}$, the target $Q_{target}$ is then frequently over-estimating (higher than the true return). This leads to the learned Q-value $Q_{\phi}$ will also become over-estimated, and the error will then propagate to all previous actions on the long-term.   
   
**Solution:** Dual DQN uses two independent value networks: (i) one to find the greedy action $a'$, (ii) one to estimate the Q-value itself. So, even if the first network chooses an over-estimated action as the greedy action, the other might provide a less over-estimated value for it, solving the problem.

**Implementation**  Applying Dual-DQN is very simple, since we already have two networks. In Step S2, just break it into two steps:
  + Greedy Action Selection by the main Q-network: change from DQN $a'=\argmax Q_{\phi'}(s_{i+1},a')$ to Dual-DQN: $a'=\argmax Q_{\phi}(s_{i+1},a')$ 
  + Estimate Q-Target in the same way: $Q_{target}= r_i + \gamma Q_{\phi'}(s_{i+1},a') $.

### 2. Priority Experienec Replay:
+ **Problem:** Another drawback of DQN is that a data batch is uniformly sampled. However, during the learning process, informative transitions are very sparse relatively to the rest of the data. Thus, uniform sampling is not very effective.  
+ **Solution:** Instead, if we can rank the data point by its level of usefulness and sample them based on its priority, we can speed up the learning. One good criterion to rank the data is using the Temporal-Difference error (TD-error), which express the supprising results we get:
  $$ \delta_i = Q_{target}(r_i,s_{i+1},a') - Q_{\phi}(s_i,a_i)$$ 
  That is, we prefer transitions that do not fit well to our current estimate of the Q function, because these are the transitions that we can learn most from. This reflects a simple intuition from our real world - if we encounter a situation that really differs from our expectation, we think about it over and over and try to solve it until it makes sense (e.g. until our model can fit it). Therefore, we update the priority of the transitions after updating the Q-networks.
+ **Implementation:** However, low $\delta$ transitions might become relevant again after enough training, as the $Q_\phi(s,a)$ change, so prioritized replay has a softmax function over the TD error to ensure “exploration” of memorized transitions.
  $$ p_i = \frac{e^{-\alpha \delta_i} }{\sum e^{-\alpha \delta_i}}$$ 

### 3. Dueling-DQN: 